# 豆瓣美国电视剧

## 只根据Tag界面筛选

### Improvement

1. 登录与非登录状态，豆瓣美剧内容显示不相同，所以该方法会有很多重复信息
   -> python 去重 还是 存入数据库 再去重 (毕竟信息不多)
   -> Spider中 Cookies 如何保存
2. Scrapy框架等的运用
3. Date提取（今年评分的很少）
   
### Progress
Oct. 7:<br/>
提取了title并插入数据库，可用。然而...爬取太快然后403 Forbidden，估计封了IP -> 不设间隔时间的爬虫都是耍流氓...<br/> 
Oct. 8:<br/>
添加rank、people、link、时间成功，可用。(评价人数不足或未上映的片子没有添加)

In [1]:
import urllib
import urllib2
import requests
import re
from lxml import etree
import time
import MySQLdb
import datetime

In [2]:
def GetPageNum(page):
    dom = etree.HTML(page)
    num = dom.xpath('//div[@class="paginator"]/a[last()]/text()')
    if len(num) > 0:
        return int(num[0])
    else:
        return 0

In [3]:
def replace(x):
    removeBlock = re.compile('\n|\n+| {1,}|<br>|&nbsp;|', re.S)
    removeBackSlash = re.compile('\/')
    x = re.sub(removeBackSlash, "", x)
    x = re.sub(removeBlock, "", x)
    return x.strip()

In [18]:
def GetContent(page, rank, people):
    dom = etree.HTML(page)
    titles = dom.xpath('//div[@class="pl2"]/a/text()')
    ratingNum = dom.xpath('//div[@class="star clearfix"]/span[@class="rating_nums"]/text()')
    peopleRaw = dom.xpath('//div[@class="star clearfix"]/span[@class="pl"]/text()')
    link = dom.xpath('//div[@class="pl2"]/a/@href')
    desp = dom.xpath('//div[@class="pl2"]/p[@class="pl"]/text()')
    
#     print len(titles), "-", len(ratingNum), "-", len(peopleRaw)

#     assert( len(ratingNum) == len(peopleRaw) )
    results = []
    seq = 0
    for title in titles:
        result = replace(title)
        if isinstance(result, unicode) == True and seq < len(ratingNum):
            movie = []
            movie.append(result)
            movie.append(float(ratingNum[seq]))
            
            pattern = re.compile('([0-9]+).*?')
            peopleNum = re.search(pattern, peopleRaw[seq])
            
            if (peopleNum) == None:
                seq += 1
                continue
            
            movie.append(int(peopleNum.group(0)))
            
            pattern = re.compile('([0-9]{4}-[0-9]{2}-[0-9]{2}).*?')
            rawTime = re.search(pattern, desp[seq])
            
            try:
                time = datetime.datetime.strptime(rawTime.group(0), '%Y-%m-%d').date()
            except:
                time = datetime.datetime.strptime("1992-12-22", '%Y-%m-%d').date() 
            
            movie.append(time)
            
            movie.append(link[seq])
            
            # print movie
            
            # print type(result), "-", type(float(ratingNum[seq])), "-", type(int(peopleNum.group(0)))
            results.append(movie)
            seq += 1
        
    return results

In [5]:
def createTable():
    conn = MySQLdb.connect(host="localhost", user="ycy", passwd="123456", db="spider", charset="utf8")
    cursor = conn.cursor()
    sql = "drop table if exists series"
    cursor.execute(sql)
    
    sql = """create table if not exists series
       (id int(10) auto_increment primary key, name varchar(300), 
       rank float, people int(10), time date, link varchar(300))
       DEFAULT CHARSET=utf8"""
    cursor.execute(sql)
    
    cursor.close()
    conn.commit()
    conn.close()

In [6]:
def WriteData(content):
    conn = MySQLdb.connect(host="localhost", user="ycy", passwd="123456", db="spider", charset="utf8")
    cursor = conn.cursor()
    
#     # 删除表
#     sql = "drop table if exists movie"
#     cursor.execute(sql)
    
    # 创建表
#     sql = """create table if not exists movie
#        (id int(10) auto_increment primary key, name varchar(1000), rank float, people int(10))
#        DEFAULT CHARSET=utf8"""
#     cursor.execute(sql)
    
#     # 测试用
#     sql = 'insert into movie(name) values("测试")'
#     n = cursor.execute(sql)
    
    # 写入多行
    sql = "insert into series(name, rank, people, time, link) values(%s, %s, %s, %s, %s)"
    n = cursor.executemany(sql, content)
#     print 'insertmany', n
    
    cursor.close()
    conn.commit()
    conn.close()

In [23]:
def Spider(url, rank, people):
    i = 0
    print "Updating... " 
    createTable()
    myPage = requests.get(url).content.decode("utf-8")
#     print myPage
    pageNum = GetPageNum(myPage)
    content = []
    print pageNum
    
    for i in range(pageNum):
        search_url = url + u"?start=" + str(i*20).decode("utf-8") + "&type=T"
#         search_url = url + u"?start=" + str(len(content)).decode("utf-8") + "&type=T"
#         search_url = url + u"?start=" + str(20).decode("utf-8") + "&type=T"
        print search_url
#         user_agent = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
#         headers = {'User-Agent': user_agent}
#         request = urllib2.Request(search_url, headers) 
#         response = urllib2.urlopen(request)
#         myPage = response.read().decode("utf-8")
        myPage = requests.get(search_url).content.decode("utf-8")
        partContent = GetContent(myPage, rank, people)
        WriteData(partContent)
        content.extend(partContent)
        print str(len(content)) + u"---------------------------------------------------------"
    
        time.sleep(20)

In [24]:
if __name__ == '__main__':
    print "start"
    start_url = "https://movie.douban.com/tag/%E7%94%B5%E8%A7%86%E5%89%A7%20%E7%BE%8E%E5%9B%BD"
#     rank = raw_input("请输入最低评分限制：\n")
#     people = raw_input("请输入最低评分人数：\n")
    rank = 8
    people = 10000
    Spider(start_url, rank, people)
    print "end"

start
Updating... 
100
https://movie.douban.com/tag/%E7%94%B5%E8%A7%86%E5%89%A7%20%E7%BE%8E%E5%9B%BD?start=0&type=T
20---------------------------------------------------------
https://movie.douban.com/tag/%E7%94%B5%E8%A7%86%E5%89%A7%20%E7%BE%8E%E5%9B%BD?start=20&type=T
38---------------------------------------------------------
https://movie.douban.com/tag/%E7%94%B5%E8%A7%86%E5%89%A7%20%E7%BE%8E%E5%9B%BD?start=40&type=T
54---------------------------------------------------------
https://movie.douban.com/tag/%E7%94%B5%E8%A7%86%E5%89%A7%20%E7%BE%8E%E5%9B%BD?start=60&type=T
72---------------------------------------------------------
https://movie.douban.com/tag/%E7%94%B5%E8%A7%86%E5%89%A7%20%E7%BE%8E%E5%9B%BD?start=80&type=T
91---------------------------------------------------------
https://movie.douban.com/tag/%E7%94%B5%E8%A7%86%E5%89%A7%20%E7%BE%8E%E5%9B%BD?start=100&type=T
109---------------------------------------------------------
https://movie.douban.com/tag/%E7%94%B5%E8%A7%86%E5%8